In [35]:
!pwd

/Users/shihosato/src/github.com/twinkle13531/master_degree/202012/Summary/p_value


In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import sys, os
sys.path.append(os.path.join('..', '..','Random'))
sys.path.append(os.path.join('..', '..', 'SA'))
sys.path.append(os.path.join('..', '..', 'QA'))
sys.path.append(os.path.join('..', '..', '4ti2', 'functions'))

In [38]:
import QA_DwaveSampler_AutoEmbedding as qda
import random_exact_test_functions as ret
import Neal_exact_test_functions as net
import zsolve_t1 as zt
import pandas as pd
import numpy as np
import timeit
import matplotlib.pyplot as plt
import statistics
import time

## t1

In [18]:
t1_dic = {}
for bit in [20, 25, 30, 35, 40, 46]:
    if bit != 46:
        t1_dic[bit] = []
        for num in [0, 1, 2, 3, 4]:
            df_path = '../../input/ost{}_{}.csv'.format(bit, num)
            df = pd.read_csv(df_path, sep=',')
            t_list = ret.calc_marginals(df)
            t1_dic[bit].append(t_list[1])
    else:
        df_path = '../../input/ost{}.csv'.format(bit)
        df = pd.read_csv(df_path, sep=',')
        t_list = ret.calc_marginals(df)
        t1_dic[46] = t_list[1]

In [19]:
t1_dic

{20: [8, 11, 10, 7, 7],
 25: [10, 8, 9, 13, 11],
 30: [12, 8, 14, 12, 12],
 35: [13, 14, 14, 15, 14],
 40: [16, 16, 15, 18, 16],
 46: 19}

## exhaustive enumeration
### {bit:{num:[valid_y_num, time, pertime, same_t1_y_num, p_value]}}

In [30]:
enu_time_dic = {}
enu_time_dic[20] = [4.94, 0.23, 2.47, 0.13, 0.08]
enu_time_dic[25] = [6.26, 1.45, 522.78, 60.29, 4.29]
enu_time_dic[30] = [114590.22, 70282.15, 4964.31, 2701.98, 594.18]

In [31]:
enu_time_dic

{20: [4.94, 0.23, 2.47, 0.13, 0.08],
 25: [6.26, 1.45, 522.78, 60.29, 4.29],
 30: [114590.22, 70282.15, 4964.31, 2701.98, 594.18]}

In [42]:
enu_valid_y_info_dic = {}
for bit in [20, 25, 30]:
    enu_valid_y_info_dic[bit] = {}
    for num in [0, 1, 2, 3, 4]:
        enu_valid_y_info_dic[bit][num] = []
        path = "../../input/ost{}_{}.csv".format(bit, num)
        df = pd.read_csv(path, sep=',')
        zinhom_path = '../../4ti2/{}bit/{}/{}bit_{}.zinhom'.format(bit, num, bit, num)
        zinhom_num, zinhom_list = zt.make_result_list(zinhom_path)
        p, num_sample = zt.calucurate_p(df, zinhom_list)
        
        enu_valid_y_info_dic[bit][num].append(num_sample)#valid_y_num
        enu_valid_y_info_dic[bit][num].append(enu_time_dic[bit][num])
        enu_valid_y_info_dic[bit][num].append(enu_time_dic[bit][num]/num_sample)
        enu_valid_y_info_dic[bit][num].append(p*num_sample)#same_t1_y_num
        enu_valid_y_info_dic[bit][num].append(p)#same_t1_y_num

In [43]:
enu_valid_y_info_dic

{20: {0: [16464, 4.94, 0.0003000485908649174, 3780.0, 0.22959183673469388],
  1: [2940, 0.23, 7.823129251700681e-05, 1225.0, 0.4166666666666667],
  2: [1050, 2.47, 0.0023523809523809526, 500.0, 0.47619047619047616],
  3: [2640, 0.13, 4.9242424242424245e-05, 1430.0, 0.5416666666666666],
  4: [2464, 0.08, 3.246753246753247e-05, 720.0, 0.2922077922077922]},
 25: {0: [12600, 6.26, 0.0004968253968253968, 1050.0, 0.08333333333333333],
  1: [5850,
   1.45,
   0.00024786324786324785,
   839.9999999999999,
   0.14358974358974358],
  2: [109010, 522.78, 0.00479570681588845, 10560.0, 0.09687184661957618],
  3: [23460, 60.29, 0.002569906223358909, 16560.0, 0.7058823529411765],
  4: [1260, 4.29, 0.003404761904761905, 168.0, 0.13333333333333333]},
 30: {0: [58296, 114590.22, 1.965661794977357, 2600.0, 0.04459997255386304],
  1: [819390, 70282.15, 0.08577374632348454, 11220.0, 0.01369311316955296],
  2: [205920, 4964.31, 0.024107954545454547, 35640.0, 0.17307692307692307],
  3: [168168, 2701.98, 0.01

## random 
### {bit:{num:[valid_y_num, time, pertime, same_t1_y_num, p_value, num_occurrence, total_valid_y]}}

In [24]:
num_reads = 10000
random_valid_y_info_dic = {}
for bit in [20, 25, 30, 35, 40, 46]:
    if bit != 46:
        random_valid_y_info_dic[bit] = {}
        for num in [0, 1, 2, 3, 4]:
            random_valid_y_info_dic[bit][num] = []
            df_path = '../../input/ost{}_{}.csv'.format(bit, num)
            df = pd.read_csv(df_path, sep=',')
            t_list = ret.calc_marginals(df)
            valid_y_dic, calculation_time = ret.find_valid_y(df, num_reads)

            valid_y_num = len(valid_y_dic)
            if valid_y_num > 0:
                random_valid_y_info_dic[bit][num].append(valid_y_num)#valid_y_num
                random_valid_y_info_dic[bit][num].append(calculation_time)
                random_valid_y_info_dic[bit][num].append(calculation_time/len(valid_y_dic))

                t1 = t_list[1]
                t1_y = 0
                for candi_y in list(valid_y_dic.keys()):
                    if np.dot(df['LI'], pd.Series(candi_y)) == t1:
                        t1_y += 1
                random_valid_y_info_dic[bit][num].append(t1_y)#same_t1_y_num
                random_valid_y_info_dic[bit][num].append(t1_y/valid_y_num)#same_t1_y_num
            
            
    else:
        random_valid_y_info_dic[bit] = []
        df_path = '../../input/ost{}.csv'.format(bit)
        df = pd.read_csv(df_path, sep=',')
        t_list = ret.calc_marginals(df)
        valid_y_dic, calculation_time = ret.find_valid_y(df, num_reads)
        
        valid_y_num = len(valid_y_dic)
        if valid_y_num > 0:
            random_valid_y_info_dic[bit].append(valid_y_num)#valid_y_num
            random_valid_y_info_dic[bit].append(calculation_time)
            random_valid_y_info_dic[bit].append(calculation_time/len(valid_y_dic))

            t1 = t_list[1]
            t1_y = 0
            for candi_y in list(valid_y_dic.keys()):
                if np.dot(df['LI'], pd.Series(candi_y)) == t1:
                    t1_y += 1
            random_valid_y_info_dic[bit].append(t1_y)#same_t1_y_num
            random_valid_y_info_dic[bit].append(t1_y/valid_y_num)#same_t1_y_num

In [25]:
random_valid_y_info_dic

{20: {0: [152,
   0.9085249900817871,
   0.0059771380926433365,
   36,
   0.23684210526315788],
  1: [25, 0.38257789611816406, 0.015303115844726562, 12, 0.48],
  2: [11, 0.3179290294647217, 0.028902639042247425, 4, 0.36363636363636365],
  3: [19, 1.074479103088379, 0.056551531741493626, 10, 0.5263157894736842],
  4: [23, 0.9098200798034668, 0.039557394774063774, 3, 0.13043478260869565]},
 25: {0: [4, 0.46073293685913086, 0.11518323421478271, 0, 0.0],
  1: [],
  2: [28, 0.904005765914917, 0.032285920211247036, 4, 0.14285714285714285],
  3: [8, 0.6662981510162354, 0.08328726887702942, 5, 0.625],
  4: [1, 0.3448147773742676, 0.3448147773742676, 0, 0.0]},
 30: {0: [],
  1: [13, 0.8195061683654785, 0.06303893602811374, 0, 0.0],
  2: [4, 0.44954395294189453, 0.11238598823547363, 0, 0.0],
  3: [4, 0.5393242835998535, 0.13483107089996338, 0, 0.0],
  4: []},
 35: {0: [],
  1: [4, 0.5745761394500732, 0.1436440348625183, 0, 0.0],
  2: [4, 0.43874287605285645, 0.10968571901321411, 0, 0.0],
  3: []

## SA 
### {bit:{num:[valid_y_num, time, pertime, same_t1_y_num, p_value, num_occurrence, total_valid_y]}}

In [44]:
num_reads = 10000
SA_valid_y_info_dic = {}
for bit in [20, 25, 30, 35, 40, 46]:
    if bit != 46:
        SA_valid_y_info_dic[bit] = {}
        for num in [0, 1, 2, 3, 4]:
            SA_valid_y_info_dic[bit][num] = []
            path = '../../input/ost{}_{}.csv'.format(bit, num)
            df = pd.read_csv(path, sep=',')

            time_0 = time.time()
            res = net.make_res_data(df, num_reads)
            valid_y_dic = net.find_valid_y(res)
            time_1 = time.time()
            calculation_time = time_1 - time_0

            valid_y_num = len(valid_y_dic)
            if valid_y_num > 0:
                SA_valid_y_info_dic[bit][num].append(valid_y_num)#valid_y_num
                SA_valid_y_info_dic[bit][num].append(calculation_time)
                SA_valid_y_info_dic[bit][num].append(calculation_time/len(valid_y_dic))

                t1 = np.dot(df['Y'], df['LI'])
                t1_y = 0
                for candi_y in list(valid_y_dic.keys()):
                    if np.dot(df['LI'], pd.Series(candi_y)) == t1:
                        t1_y += 1
                SA_valid_y_info_dic[bit][num].append(t1_y)#same_t1_y_num
                SA_valid_y_info_dic[bit][num].append(t1_y/valid_y_num)#same_t1_y_num
    else:
        SA_valid_y_info_dic[bit] = []
        path = '../../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')

        time_0 = time.time()
        res = net.make_res_data(df, num_reads)
        valid_y_dic = net.find_valid_y(res)
        time_1 = time.time()
        calculation_time = time_1 - time_0

        valid_y_num = len(valid_y_dic)
        if valid_y_num > 0:
            SA_valid_y_info_dic[bit].append(valid_y_num)#valid_y_num
            SA_valid_y_info_dic[bit].append(calculation_time)
            SA_valid_y_info_dic[bit].append(calculation_time/len(valid_y_dic))

            t1 = np.dot(df['Y'], df['LI'])
            t1_y = 0
            for candi_y in list(valid_y_dic.keys()):
                if np.dot(df['LI'], pd.Series(candi_y)) == t1:
                    t1_y += 1
            SA_valid_y_info_dic[bit].append(t1_y)#same_t1_y_num
            SA_valid_y_info_dic[bit].append(t1_y/valid_y_num)#same_t1_y_num

FileNotFoundError: [Errno 2] File ../../input/ost46_4.csv does not exist: '../../input/ost46_4.csv'

In [ ]:
SA_valid_y_info_dic

## QA 
### {bit:{num:[valid_y_num, time, pertime, same_t1_y_num, p_value, num_occurrence, total_valid_y, chain_strength, broken_percent]}}

In [ ]:
num_reads = 10000
random_valid_y_info_dic = {}
for bit in [20, 25, 30, 35, 40]:
    
    random_valid_y_info_dic[bit] = {}
    for num in [0, 1, 2, 3, 4]:
        random_valid_y_info_dic[bit][num] = []
        df_path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(df_path, sep=',')
        t_list = ret.calc_marginals(df)
        valid_y_dic, calculation_time = ret.find_valid_y(df, num_reads)
        
        valid_y_num = len(valid_y_dic)
        if valid_y_num > 0:
            random_valid_y_info_dic[bit][num].append(valid_y_num)#valid_y_num

            t1 = t_list[1]
            t1_y = 0
            for candi_y in list(valid_y_dic.keys()):
                if np.dot(df['LI'], pd.Series(candi_y)) == t1:
                    t1_y += 1
            random_valid_y_info_dic[bit][num].append(t1_y)#same_t1_y_num
            random_valid_y_info_dic[bit][num].append(t1_y/valid_y_num)#same_t1_y_num